## Importing dependencies and environmental variables

In [1]:
# Dependencies
import csv as csv
import http.client 
import json
import numpy as np
import os
import pandas as pd
import pprint
import psycopg2
import requests
import sqlalchemy
from bs4 import BeautifulSoup as bs
from dotenv import load_dotenv
from selenium import webdriver
from sodapy import Socrata
from splinter import Browser
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, Float, Date

In [2]:
# Loading homeowrk7.env so that the environment variables can be used in the engine object url
load_dotenv("project3.env")
# Initializing variables to hold each environmet varaible
username=os.environ.get("USERNAME")
password=os.environ.get("PASSWORD")
host=os.environ.get("HOST")
port=os.environ.get("PORT")
database=os.environ.get("DATABASE")
google_key = os.environ.get('GOOGLE_API_KEY')
yelp_client_id = os.environ.get('YELP_CLIENT_ID')
yelp_key = os.environ.get('YELP_API_KEY')
sf_data_key = os.environ.get('SFDATAAPPTOKEN')

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

## Querying the Los Angeles Active Business API for Startbucks locations

In [16]:
client = Socrata("data.lacity.org", sf_data_key)

In [17]:
starbucks_results = client.get("6rrh-rzua", where="business_name LIKE '%STARBUCKS%'")
starbucks_results_df = pd.DataFrame.from_records(starbucks_results)

In [18]:
starbucks_results_df

,business_name,city,council_district,dba_name,location_1,location_account,location_description,location_start_date,mailing_address,mailing_city,mailing_zip_code,naics,primary_naics_description,street_address,zip_code
0,STARBUCKS CORPORATION,MISSION HILLS,7,STARBUCKS COFFEE #5833,"{'latitude': '34.2573', 'longitude': '-118.4696'}",0000400401-0001-8,15501 DEVONSHIRE 91345-2620,2001-02-03T00:00:00.000,POST OFFICE BOX #34442,SEATTLE,98124-1442,445290,Home furnishings stores,15501 DEVONSHIRE STREET,91345-2620
1,STARBUCKS CORPORATION,WOODLAND HILLS,3,STARBUCKS COFFEE #18869,"{'latitude': '34.1576', 'longitude': '-118.6353'}",0000008207-0011-2,23355 MULHOLLAND 91364-2734,2013-04-04T00:00:00.000,POST OFFICE BOX #34442,SEATTLE,98124-1442,445290,Home furnishings stores,23355 MULHOLLAND DRIVE,91364-2734
2,STARBUCKS CORPORATION,LOS ANGELES,14,STARBUCKS COFFEE #24871,"{'latitude': '34.0488', 'longitude': '-118.2517'}",0000008207-0032-5,401 5TH 90013-1001,2015-07-02T00:00:00.000,POST OFFICE BOX #34442,SEATTLE,98124-1442,311900,"Other food mfg. (including coffee, tea, flavor...",401 W 5TH STREET,90013-1001
3,STARBUCKS CORPORATION,PACIFIC PALISADES,11,STARBUCKS COFFEE #609,"{'latitude': '34.0474', 'longitude': '-118.5257'}",0000155172-0001-5,15300 SUNSET 90272-3613,1994-10-15T00:00:00.000,POST OFFICE BOX #34442,SEATTLE,98124-1442,445290,Home furnishings stores,15300 W SUNSET BLVD,90272-3613
4,STARBUCKS CORPORATION,NORTH HILLS,7,NaN,"{'latitude': '34.2335', 'longitude': '-118.4674'}",0000024503-0002-3,9000 SEPULVEDA 91343-4308,2005-06-03T00:00:00.000,POST OFFICE BOX #34442,SEATTLE,98124-1442,445290,Home furnishings stores,9000 SEPULVEDA BLVD #8637,91343-4308
5,STARBUCKS CORPORATION,LOS ANGELES,13,STARBUCKS COFFEE #55650,"{'latitude': '34.0978', 'longitude': '-118.2921'}",0002363980-0015-7,4700 SUNSET 90027-6082,2018-09-27T00:00:00.000,POST OFFICE BOX #34442,SEATTLE,98124-1442,311900,"Other food mfg. (including coffee, tea, flavor...",4700 W SUNSET BLVD,90027-6082
6,STARBUCKS CORPORATION,STUDIO CITY,2,STARBUCKS COFFEE #573,"{'latitude': '34.1433', 'longitude': '-118.3975'}",0000115140-0001-0,12170 VENTURA 91604-2514,1994-07-01T00:00:00.000,POST OFFICE BOX #34442,SEATTLE,98124-1442,445290,Home furnishings stores,12170 VENTURA BLVD,91604-2514
7,STARBUCKS CORPORATION,LOS ANGELES,9,STARBUCKS COFFEE #21800,"{'latitude': '33.9882', 'longitude': '-118.2564'}",0000008207-0018-0,5857 CENTRAL 90001-1127,2014-03-14T00:00:00.000,POST OFFICE BOX #34442,SEATTLE,98124-1442,311900,"Other food mfg. (including coffee, tea, flavor...",5857 S CENTRAL AVENUE,90001-1127
8,STARBUCKS CORPORATION,LOS ANGELES,5,STARBUCKS COFFEE #546,"{'latitude': '34.0394', 'longitude': '-118.4298'}",0000072045-0001-5,10911 PICO 90064-2111,1993-03-27T00:00:00.000,POST OFFICE BOX #34442,SEATTLE,98124-1442,445290,Home furnishings stores,10911 W PICO BLVD,90064-2111
9,STARBUCKS CORPORATION,LOS ANGELES,4,STARBUCKS COFFEE CO #507,"{'latitude': '34.0747', 'longitude': '-118.3236'}",0000860517-0001-7,206 LARCHMONT 90004-3707,1992-01-06T00:00:00.000,POST OFFICE BOX #34442,SEATTLE,98124-1442,445290,Home furnishings stores,206 N LARCHMONT BLVD,90004-3707


## Querying the Los Angeles Active Business API for bourgie coffee shops

In [70]:
bourgie_coffee_shop_list = ["ALFRED COFFEE",
                            "BLUE BOTTLE COFFEE",
                            "PEETS COFFEE",
                            "COFFEE COMMISSARY",
                            "COGNOSCENTI COFFEE",
                            "ENDORFFEINE",
                            "G&B COFFEE",
                            "MENOTTI",
                            "GO GET EM TIGER",
                            "PARAMOUNT COFFEE PROJECT",
                            "RUBIES AND DIAMONDS",
                            "VERVE COFFEE ROASTERS",
                            "FOUND COFFEE",
                            "THE BOY & THE BEAR - COFFEE ROASTERY",
                            "HILLTOP COFFEE + KITCHEN",
                            "TACTILE COFFEE",
                            "NOSSA FAMILIA COFFEE",
                            "TIERRA MIA COFFEE COMPANY"
                           ]

In [71]:
bourgie_coffee_df = pd.DataFrame()
for coffee_shop in bourgie_coffee_shop_list:
    query = client.get("6rrh-rzua", where=f"dba_name LIKE '%{coffee_shop}%'")
    results_df = pd.DataFrame.from_records(query)
    bourgie_coffee_df = pd.concat([bourgie_coffee_df, results_df], ignore_index=True, sort=True)

## Concatenating all coffee shop data frames together and cleaning them

In [72]:
complete_coffee_df = pd.DataFrame()
complete_coffee_df = pd.concat([starbucks_results_df, 
                                bourgie_coffee_df
                               ], ignore_index=True, sort=True)

In [73]:
complete_coffee_df

,business_name,city,council_district,dba_name,location_1,location_account,location_description,location_start_date,mailing_address,mailing_city,mailing_zip_code,naics,primary_naics_description,street_address,zip_code
0,STARBUCKS CORPORATION,MISSION HILLS,7,STARBUCKS COFFEE #5833,"{'latitude': '34.2573', 'longitude': '-118.4696'}",0000400401-0001-8,15501 DEVONSHIRE 91345-2620,2001-02-03T00:00:00.000,POST OFFICE BOX #34442,SEATTLE,98124-1442,445290,Home furnishings stores,15501 DEVONSHIRE STREET,91345-2620
1,STARBUCKS CORPORATION,WOODLAND HILLS,3,STARBUCKS COFFEE #18869,"{'latitude': '34.1576', 'longitude': '-118.6353'}",0000008207-0011-2,23355 MULHOLLAND 91364-2734,2013-04-04T00:00:00.000,POST OFFICE BOX #34442,SEATTLE,98124-1442,445290,Home furnishings stores,23355 MULHOLLAND DRIVE,91364-2734
2,STARBUCKS CORPORATION,LOS ANGELES,14,STARBUCKS COFFEE #24871,"{'latitude': '34.0488', 'longitude': '-118.2517'}",0000008207-0032-5,401 5TH 90013-1001,2015-07-02T00:00:00.000,POST OFFICE BOX #34442,SEATTLE,98124-1442,311900,"Other food mfg. (including coffee, tea, flavor...",401 W 5TH STREET,90013-1001
3,STARBUCKS CORPORATION,PACIFIC PALISADES,11,STARBUCKS COFFEE #609,"{'latitude': '34.0474', 'longitude': '-118.5257'}",0000155172-0001-5,15300 SUNSET 90272-3613,1994-10-15T00:00:00.000,POST OFFICE BOX #34442,SEATTLE,98124-1442,445290,Home furnishings stores,15300 W SUNSET BLVD,90272-3613
4,STARBUCKS CORPORATION,NORTH HILLS,7,NaN,"{'latitude': '34.2335', 'longitude': '-118.4674'}",0000024503-0002-3,9000 SEPULVEDA 91343-4308,2005-06-03T00:00:00.000,POST OFFICE BOX #34442,SEATTLE,98124-1442,445290,Home furnishings stores,9000 SEPULVEDA BLVD #8637,91343-4308
5,STARBUCKS CORPORATION,LOS ANGELES,13,STARBUCKS COFFEE #55650,"{'latitude': '34.0978', 'longitude': '-118.2921'}",0002363980-0015-7,4700 SUNSET 90027-6082,2018-09-27T00:00:00.000,POST OFFICE BOX #34442,SEATTLE,98124-1442,311900,"Other food mfg. (including coffee, tea, flavor...",4700 W SUNSET BLVD,90027-6082
6,STARBUCKS CORPORATION,STUDIO CITY,2,STARBUCKS COFFEE #573,"{'latitude': '34.1433', 'longitude': '-118.3975'}",0000115140-0001-0,12170 VENTURA 91604-2514,1994-07-01T00:00:00.000,POST OFFICE BOX #34442,SEATTLE,98124-1442,445290,Home furnishings stores,12170 VENTURA BLVD,91604-2514
7,STARBUCKS CORPORATION,LOS ANGELES,9,STARBUCKS COFFEE #21800,"{'latitude': '33.9882', 'longitude': '-118.2564'}",0000008207-0018-0,5857 CENTRAL 90001-1127,2014-03-14T00:00:00.000,POST OFFICE BOX #34442,SEATTLE,98124-1442,311900,"Other food mfg. (including coffee, tea, flavor...",5857 S CENTRAL AVENUE,90001-1127
8,STARBUCKS CORPORATION,LOS ANGELES,5,STARBUCKS COFFEE #546,"{'latitude': '34.0394', 'longitude': '-118.4298'}",0000072045-0001-5,10911 PICO 90064-2111,1993-03-27T00:00:00.000,POST OFFICE BOX #34442,SEATTLE,98124-1442,445290,Home furnishings stores,10911 W PICO BLVD,90064-2111
9,STARBUCKS CORPORATION,LOS ANGELES,4,STARBUCKS COFFEE CO #507,"{'latitude': '34.0747', 'longitude': '-118.3236'}",0000860517-0001-7,206 LARCHMONT 90004-3707,1992-01-06T00:00:00.000,POST OFFICE BOX #34442,SEATTLE,98124-1442,445290,Home furnishings stores,206 N LARCHMONT BLVD,90004-3707


In [74]:
complete_coffee_df.to_csv("data/los_angeles_coffee_shop_data.csv", index=False, header=True)